In [16]:
# Se cargan las librerías y tabla de datos
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
import os
# os.getcwd()
os.chdir('/home/clautc/Proyectos_/CrianzasChile')
from plotly.subplots import make_subplots
n = []
for x in range(1,8):
    n.append(x)
nombres =['_jefatura_menor18',
          '_quintil_menor_18',
          '_pobreza_menor_18',
          '_thogar_menor_18',
          '_thogar_jefatura_18',
          '_thogar_pobreza_18',
          '_thogar_quintil_18'
          ]

# nombres =['_15',
#           '_15',
#           '_15',
#           '_15',
#           '_15',
#           '_15',
#           '_15'
#           ]

rutas = []
for s in n:
    ruta = 'resultados/tablas/0' + str(s) + nombres[s-1] + '.csv'
    rutas.append(ruta)

rutas

['resultados/tablas/01_jefatura_menor18.csv',
 'resultados/tablas/02_quintil_menor_18.csv',
 'resultados/tablas/03_pobreza_menor_18.csv',
 'resultados/tablas/04_thogar_menor_18.csv',
 'resultados/tablas/05_thogar_jefatura_18.csv',
 'resultados/tablas/06_thogar_pobreza_18.csv',
 'resultados/tablas/07_thogar_quintil_18.csv']

In [17]:
def grafico_agrupado_2x(data, var_estimacion, valor_estimacion, var_trazas, vars_x, var_y, var_ic_bajo, var_ic_alto,
                        title_yaxes, title, nombre_graf):
    df = data.copy()
    fig = go.Figure()
    trazas = df.loc[df[var_estimacion] == valor_estimacion, var_trazas].unique()
    for t in trazas:
        x = [df.loc[(df[var_estimacion] == valor_estimacion) & (df[var_trazas] == t), vars_x[0]].to_list(),
             df.loc[(df[var_estimacion] == valor_estimacion) & (df[var_trazas]  == t), vars_x[1]].to_list()]
        y = df.loc[(df[var_estimacion] == valor_estimacion) & (df[var_trazas]  == t), var_y]
        array = df.loc[(df[var_estimacion] == valor_estimacion) & (df[var_trazas]  == t), var_ic_alto]
        arraymin = df.loc[(df[var_estimacion] == valor_estimacion) & (df[var_trazas]  == t), var_ic_bajo]
        fig.add_trace(
            go.Bar(x = x,
                   y = y,
                   name=t,
                   error_y={
                       "type": "data",
                       "symmetric": False,
                       "array": array,
                       "arrayminus": arraymin,
                       "color": 'black',
                       "thickness":3,
                       "width":6
                   }
                   )

        )
        fig.update_layout(barmode='group')
        fig.update_yaxes(title=title_yaxes)
        fig.update_layout(title=title,
                          template='none')

    final = 'resultados/img/' + nombre_graf + '.jpeg'
    fig.write_image(final)
    fig.show()

In [18]:
def cambio_cat_estimacion(df01):
    df01.loc[df01.tipo_est == 'conteo', 'tipo_est'] = 'proporcion'
    df01.loc[df01.tipo_est != 'proporcion', 'tipo_est']  = 'conteo'
    return df01

In [19]:
df01 = pd.read_csv(rutas[0])
# df01.loc[df01.tipo_est == 'conteo', 'tipo_est'] = 'proporcion'
# df01.loc[df01.tipo_est != 'proporcion', 'tipo_est']  = 'conteo'
grafico_agrupado_2x(
    data=df01,
    var_estimacion='tipo_est',
    valor_estimacion='proporcion',
    var_y='dato',
    var_trazas='menor_18',
    vars_x=['anio', 'sexo_jefatura'],
    var_ic_alto='ic_alto',
    var_ic_bajo='ic_bajo',
    title='<b>Distribución de hogares con y sin presencia de personas menores de 18 años,<br>según '
          'sexo de la jefatura de hogar',
    title_yaxes='Total hogares (%)',
nombre_graf='02_01')
print(df01.head())

   Unnamed: 0 sexo_jefatura     menor_18       dato  desviacion_s  \
0           0        hombre  sin menores   717265.0  11032.115748   
1           1        hombre  con menores  1821683.0  14877.663581   
2           2         mujer  sin menores   305118.0   7757.556011   
3           3         mujer  con menores   336470.0   7784.102512   
4           4        hombre  sin menores       22.6      0.003287   

        ic_bajo       ic_alto    tipo_est  anio  
0  6.956414e+05  7.388886e+05      conteo  1990  
1  1.792522e+06  1.850844e+06      conteo  1990  
2  2.899128e+05  3.203232e+05      conteo  1990  
3  3.212127e+05  3.517273e+05      conteo  1990  
4  2.191390e-01  2.320256e-01  proporcion  1990  


In [20]:
fig = px.bar(df01.loc[df01.tipo_est == 'proporcion'],
             x =  'sexo_jefatura' ,
             y = 'dato', facet_col= 'anio',
             color = 'menor_18',
             facet_col_wrap=6,
             labels = {
                 'menor_18': '<b>Presencia de <br>menores</b>',
                 'dato': 'Hogares(%)',
                 # 'pobreza': None
                 # 'anio': 'Periodo'
             },
             error_y_minus='ic_bajo',
             error_y='ic_alto',
             # error_y={
             #     "type": "data",
             #     "symmetric": False,
             #     "array": 'ic_alto',
             #     "arrayminus": 'ic_bajo',
             #     "color": 'black',
             #     "thickness":3,
             #     "width":2
             # },
             # category_orders={
             #     'pobreza': ['no pobre','pobre no extremo', 'pobre extremo']
             # },

             color_discrete_map={
                 'sin menores': '#2471A3',
                 'con menores': '#E67E22'
             },
             text ='dato'


             )
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]),
                        # x=0.2, xref='x domain',
                        # y=0.8, yref='y domain'
                        )
fig.update_layout(template='plotly',
                  # title ='<b>Evolución de la distribución de hogares con y sin menores de 18 años, según situación de '
                  #        'pobreza</b>',
                  height=600,
                  # width=1100
                  margin=
                  dict(
                      l=20,
                      r=20,
                      b=20,
                      t=20
                  ),
                  barmode='group',
                  legend=dict(
                      # title=None,
                      orientation='h', y=1.02, yanchor='bottom', x=0.2, xanchor='center'
                  )
                  )
fig.update_xaxes(title=None,
                 showgrid=True,
                 tickangle=0,


                 )
fig.update_yaxes(
    title_font_size = 10,
    title_standoff = 8
)
fig.write_image('resultados/img/02_01_px.jpeg')

fig.show()

In [21]:
df02 = pd.read_csv(rutas[1])
# df02 = cambio_cat_estimacion(df02)
grafico_agrupado_2x(
    data=df02,
    var_estimacion='tipo_est',
    valor_estimacion='proporcion',
    var_y='dato',
    var_trazas='menor_18',
    vars_x=['anio', 'quintil'],
    var_ic_alto='ic_alto',
    var_ic_bajo='ic_bajo',
    title='<b>Distribución de hogares con y sin presencia de personas menores de 18 años,<br>según '
          'quintiles de ingreso',
    title_yaxes='Total hogares (%)',
    nombre_graf='02_02'
)
# df02.head(20)

In [35]:
df02 = pd.read_csv(rutas[1])
fig = px.bar(df02.loc[df02.tipo_est == 'proporcion'],
             x =  'quintil' ,
             y = 'dato', facet_col= 'anio',
             color = 'menor_18',
             facet_col_wrap=6,
             labels = {
                 'menor_18': '<b>Presencia de <br>menores</b>',
                 'dato': 'Hogares(%)',
                 # 'pobreza': None
                 # 'anio': 'Periodo'
             },
             error_y_minus='ic_bajo',
             error_y='ic_alto',
             # error_y={
             #     "type": "data",
             #     "symmetric": False,
             #     "array": 'ic_alto',
             #     "arrayminus": 'ic_bajo',
             #     "color": 'black',
             #     "thickness":3,
             #     "width":2
             # },
             # category_orders={
             #     'pobreza': ['no pobre','pobre no extremo', 'pobre extremo']
             # },

             color_discrete_map={
                 'sin menores': '#2471A3',
                 'con menores': '#E67E22'
             },
             text ='dato'


             )
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]),
                        # x=0.2, xref='x domain',
                        # y=0.8, yref='y domain'
                        )
fig.update_layout(template='plotly',
                  # title ='<b>Evolución de la distribución de hogares con y sin menores de 18 años, según situación de '
                  #        'pobreza</b>',
                  height=600,
                  # width=1100
                  margin=
                  dict(
                      l=20,
                      r=20,
                      b=20,
                      t=20
                  ),
                  barmode='group',
                  legend=dict(
                      # title=None,
                      orientation='h', y=1.02, yanchor='bottom', x=0.2, xanchor='center'
                  )
                  )
fig.update_xaxes(title=None,
                 showgrid=True,
                 tickangle=0,


                 )
fig.update_yaxes(
    title_font_size = 10,
    title_standoff = 8
)
fig.write_image('resultados/img/02_02_px.jpeg')

fig.show()

In [23]:
df03 = pd.read_csv(rutas[2])
# grafico_agrupado_2x(
#     data=df03,
#     var_estimacion='tipo_est',
#     valor_estimacion='proporcion',
#     var_y='dato',
#     var_trazas='menor_18',
#     vars_x=['anio', 'pobreza'],
#     var_ic_alto='ic_alto',
#     var_ic_bajo='ic_bajo',
#     title='<b>Distribución de hogares con y sin presencia de personas menores de 18 años,<br>según '
#           'pobreza por ingreso',
#     title_yaxes='Total hogares (%)',
#     nombre_graf='02_03'
# )
# # df03.head(10)

In [25]:
fig = px.bar(df03.loc[df03.tipo_est == 'proporcion'],
             x =  'pobreza',
             y = 'dato', facet_col= 'anio',
             color = 'menor_18',
             facet_col_wrap=3,
             labels = {
                 'menor_18': '<b>Presencia de <br>menores</b>',
                 'dato': 'Hogares(%)',
                 # 'pobreza': None
                 # 'anio': 'Periodo'
             },
             error_y_minus='ic_bajo',
             error_y='ic_alto',

             category_orders={
                 'pobreza': ['no pobre', 'pobre no extremo', 'pobre extremo']
             },

             color_discrete_map={
                 'sin menores': '#2471A3',
                 'con menores': '#E67E22'
             },
             text ='dato'


             )
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]),

                        )
fig.update_layout(template='plotly',
                  # title ='<b>Evolución de la distribución de hogares con y sin menores de 18 años, según situación de '
                  #        'pobreza</b>',
                  height=600,
                  width=1600,

                  margin=
                  dict(
                      l=5,
                      r=5,
                      b=20,
                      t=15
                  ),
                  barmode='group',
                  legend=dict(
                      # title=None,
                      orientation='h', y=1, yanchor='bottom', x=0.2, xanchor='center'
                  )
                  )
fig.update_xaxes(title=None,
                 showgrid=True,
                 tickangle=0,


                 )
fig.update_yaxes(
    title_font_size = 10,
    title_standoff = 8
)
fig.write_image('resultados/img/02_03_px.jpeg')

fig.show()

In [26]:
df04 = pd.read_csv(rutas[3])
fig = px.bar(df04.loc[df04.tipo_est == 'proporcion'],
             x =  'thogar' ,
             y = 'dato', facet_row= 'anio',
             color = 'menor_18',
             # facet_col_wrap=2,
             labels = {
                 'menor_18': '<b>Presencia de <br>menores</b>',
                 'dato': 'Hogares(%)',
                 # 'pobreza': None
                 # 'anio': 'Periodo'
             },
             error_y_minus='ic_bajo',
             error_y='ic_alto',

             color_discrete_map={
                 'sin menores': '#2471A3',
                 'con menores': '#E67E22'
             },
             text ='dato'


             )
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]),

                        )
fig.update_layout(template='plotly',

                  height=900,
                  # width=1100
                  margin=
                  dict(
                      l=10,
                      r=10,
                      b=10,
                      t=25
                  ),
                  barmode='group',
                  legend=dict(
                      # title=None,
                      orientation='h', y=1, yanchor='bottom', x=0.2, xanchor='center'
                  )
                  )
fig.update_xaxes(title=None,
                 showgrid=True,
                 tickangle=0,


                 )
fig.update_yaxes(
    title_font_size = 10,
    title_standoff = 8
)
fig.write_image('resultados/img/02_04_px.jpeg')
fig.show()

In [27]:
df05 = pd.read_csv(rutas[4])
fig = px.bar(df05.loc[df05.tipo_est == 'proporcion'],
             x =  'thogar' ,
             y = 'dato', facet_row= 'anio',
             color = 'sexo_jefatura',
             # facet_col_wrap=2,
             labels = {
                 'sexo_jefatura': '<b>Sexo de jefatura del hogar</b>',
                 'dato': 'Hogares(%)',
                 # 'pobreza': None
                 # 'anio': 'Periodo'
             },
             error_y_minus='ic_bajo',
             error_y='ic_alto',

             color_discrete_map={
                 'sin menores': '#2471A3',
                 'con menores': '#E67E22'
             },
             text ='dato'


             )
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]),

                        )
fig.update_layout(template='plotly',

                  height=900,
                  # width=1100
                  margin=
                  dict(
                      l=10,
                      r=10,
                      b=10,
                      t=15
                  ),
                  barmode='group',
                  legend=dict(
                      # title=None,
                      orientation='h', y=1, yanchor='bottom', x=0.2, xanchor='center'
                  )
                  )
fig.update_xaxes(title=None,
                 showgrid=True,
                 tickangle=0,


                 )
fig.update_yaxes(
    title_font_size = 10,
    title_standoff = 8
)
fig.write_image('resultados/img/02_05_px.jpeg')
fig.show()

In [28]:
df06 = pd.read_csv(rutas[5])

fig = px.bar(df06.loc[df06.tipo_est == 'proporcion'],
             x =  'thogar' ,
             y = 'dato', facet_row= 'anio',
             color = 'pobreza',
             # facet_col_wrap=2,
             labels = {
                 '': '<b>Situación de pobreza</b>',
                 'dato': 'Hogares(%)',
                 # 'pobreza': None
                 # 'anio': 'Periodo'
             },
             error_y_minus='ic_bajo',
             error_y='ic_alto',

             color_discrete_map={
                 'sin menores': '#2471A3',
                 'con menores': '#E67E22'
             },
             text ='dato'


             )
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]),

                        )
fig.update_layout(template='plotly',

                  height=900,
                  # width=1100
                  margin=
                  dict(
                      l=10,
                      r=10,
                      b=10,
                      t=15
                  ),
                  barmode='group',
                  legend=dict(
                      # title=None,
                      orientation='h', y=1, yanchor='bottom', x=0.2, xanchor='center'
                  )
                  )
fig.update_xaxes(title=None,
                 showgrid=True,
                 tickangle=0,


                 )
fig.update_yaxes(
    title_font_size = 10,
    title_standoff = 8
)
fig.write_image('resultados/img/02_06_px.jpeg')
fig.show()

In [29]:
df07 = pd.read_csv(rutas[6])
fig = px.bar(df07.loc[df07.tipo_est == 'proporcion'],
             x =  'thogar' ,
             y = 'dato', facet_row= 'anio',
             color = 'quintil',
             # facet_col_wrap=2,
             labels = {
                 'quintil': '<b>Quintiles de ingreso</b>',
                 'dato': 'Hogares(%)',
                 # 'pobreza': None
                 # 'anio': 'Periodo'
             },
             error_y_minus='ic_bajo',
             error_y='ic_alto',

             color_discrete_map={
                 'sin menores': '#2471A3',
                 'con menores': '#E67E22'
             },
             text ='dato'


             )
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]),

                        )
fig.update_layout(template='plotly',

                  height=900,
                  # width=1100
                  margin=
                  dict(
                      l=10,
                      r=10,
                      b=10,
                      t=15
                  ),
                  barmode='group',
                  legend=dict(
                      # title=None,
                      orientation='h', y=1, yanchor='bottom', x=0.2, xanchor='center'
                  )
                  )
fig.update_xaxes(title=None,
                 showgrid=True,
                 tickangle=0,


                 )
fig.update_yaxes(
    title_font_size = 10,
    title_standoff = 8
)
fig.write_image('resultados/img/02_07_px.jpeg')
fig.show()

In [30]:
fig = px.bar(df07.loc[df07.tipo_est == 'proporcion'],
             x =  'quintil'  ,
             y = 'dato', facet_col= 'anio' ,
             color ='thogar',
             facet_col_wrap=6,
             error_y_minus='ic_bajo',
             error_y='ic_alto',
             labels = {
                 'thogar': '<b>Tipos hogar</b>',
                 'dato': 'Hogares(%)',
                 # 'pobreza': None
                 # 'anio': 'Periodo'
             },
             # category_orders={
             #     'pobreza': ['no pobre','pobre no extremo', 'pobre extremo']
             # },
             # color_discrete_map={
             #     'sin menores': '#2471A3',
             #     'con menores': '#E67E22'
             # },
             text ='dato'
             )
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]),
                        # x=0.2, xref='x domain',
                        # y=0.8, yref='y domain'
                        )
fig.update_layout(template='plotly',
                  title ='<b>Distribución de tipología estructural de hogares, según quintiles de ingresos',
                  height=600,
                  # width=1100
                  margin=
                  dict(
                      t=250
                  ),
                  barmode='group',
                  legend=dict(
                      # title=None,
                      orientation='h', y=1.2, yanchor='bottom', x=0.4, xanchor='center'
                  )
                  )
fig.update_xaxes(title=None,
                 showgrid=True,
                 tickangle=90)
fig.update_yaxes( title_font_size = 10,
                  title_standoff = 8
                  )
fig.write_image('resultados/img/02_07.jpeg')
fig.show()

In [33]:
# print(6.3/0.9)
# 7.2/3.3
print(9.5/0.8)
6.9/3

11.875


2.3000000000000003